<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/s2sattn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive


In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math
from keras.optimizers import SGD
from keras.initializers import RandomUniform
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed, Bidirectional, Embedding
from keras.layers import RepeatVector
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation
from keras.layers import dot, concatenate
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [115]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['src'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest'])
lines = pd.concat([X[:3000],Y[:3000]], axis=1)
print(len(lines))
# lines = shuffle(lines)
lines.sample(10)

3000


,src,dest
2456,The infant must be well taken care of at the t...,जन्म के समय शिशु की देखभाल भली-भाँति की जानी च...
1166,Life can be in danger if the treatment of hern...,हर्निया का इलाज न करवाने से जान खतरे में पड़ सक...
988,Every fifth person in India is a heart patient .,भारत में हर पाँचवाँ व्यक्ति दिल का मरीज है ।
1258,Desire has control on brain and desire works a...,मस्तिष्क पर इच्छा का नियंत्रण होता है और इच्छा...
802,By adopting good habits of proper medical care...,"उचित चिकित्सीय देखभाल , शल्यचिकित्सीय विकल्पों..."
505,The following directions could be helpful in s...,नीचे दिये गये निर्देश यौन जनित रोगों के बचाव म...
683,Sometimes this amoeba enters the portal vein a...,`` दिन में 12 से लेकर 24 तथा उससे भी अधिक बार ...
2854,Due to swelling in the mucous membrane of brea...,श्‍वास नलियों की श्‍लेष्मा झिल्ली में सूजन आने...
1585,In some parts of America and Africa beans is c...,अमेरिका तथा अफ्रीका के कुछ भाग में तो बीन्स को...
433,Encouraging the patient to change emotionally .,रोगी को अपने भावनात्मक रूप से बदलाव के लिऐ प्र...


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : '<SOS> '+ x + ' <EOS>')
  return data

In [0]:
lines.src = process_data(lines.src,False)
lines.dest = process_data(lines.dest,True)
# lines.dest.sample(10)

In [0]:
def get_vocab_list(data):
  vocab = []
  for line in data:
    temp = line.split()
    for item in temp:
      if item not in vocab:
        vocab.append(item)
  # print(vocab)
  return vocab

In [0]:
src_vocab = get_vocab_list(lines.src)
dest_vocab = get_vocab_list(lines.dest)

In [120]:
print(src_vocab[:10])
print(dest_vocab[:10]) 

['the', 'treatment', 'of', 'cataract', 'is', 'possible', 'through', 'surgery', 'only', 'complete']
['<SOS>', 'मोतियाबिंद', 'का', 'उपचार', 'केवल', 'शल्यचिकित्सा', 'द्वारा', 'ही', 'सम्भव', 'है']


In [0]:
# Max Length of source sequence
def get_max_sentence_size(data):
  max_len = 0
  for line in data:
    temp = line.split()
    max_len = max(max_len, len(temp))
  return max_len

In [122]:
src_max_len = get_max_sentence_size(lines.src)
dest_max_len = get_max_sentence_size(lines.dest)
src_max_len,dest_max_len

(84, 86)

In [123]:
src_vocab_len = len(src_vocab)
dest_vocab_len = len(dest_vocab) + 1
src_vocab_len, dest_vocab_len

(4711, 6289)

In [0]:
def get_word_to_index_dict(vocab):
  word_to_index = {}
  for i, word in enumerate(vocab):
    word_to_index[word] = i
  return word_to_index

In [0]:
src_word_to_index = get_word_to_index_dict(src_vocab)
dest_word_to_index = get_word_to_index_dict(dest_vocab)

In [126]:
print(src_word_to_index)
print(dest_word_to_index)

{'the': 0, 'treatment': 1, 'of': 2, 'cataract': 3, 'is': 4, 'possible': 5, 'through': 6, 'surgery': 7, 'only': 8, 'complete': 9, 'lens': 10, 'capsule': 11, 'taken': 12, 'out': 13, 'in': 14, 'method': 15, 'intra': 16, 'capsular': 17, 'extraction': 18, 'after': 19, 'operation': 20, 'spectacles': 21, 'are': 22, 'used': 23, 'posttest': 24, 'during': 25, 'implanted': 26, 'at': 27, 'front': 28, 'iris': 29, 'frontal': 30, 'part': 31, 'eye': 32, 'extra': 33, 'and': 34, 'whole': 35, 'matter': 36, 'extracted': 37, 'posterior': 38, 'left': 39, 'intact': 40, 'fitted': 41, 'bag': 42, 's': 43, 'lrb': 44, 'small': 45, 'incision': 46, 'rrb': 47, 'mm': 48, 'tunnel': 49, 'formed': 50, 'while': 51, 'rest': 52, 'follows': 53, 'like': 54, 'that': 55, 'ecceiol': 56, 'no': 57, 'stitches': 58, 'applied': 59, 'black': 60, 'nerves': 61, 'dilapidates': 62, 'gradually': 63, 'it': 64, 'happens': 65, 'due': 66, 'to': 67, 'water': 68, 'pressure': 69, 'or': 70, 'because': 71, 'not': 72, 'enough': 73, 'blood': 74, 're

In [0]:
def get_index_to_word_dict(vocab):
  index_to_word = {}
  for i, word in enumerate(vocab):
    index_to_word[i] = word
  return index_to_word  

In [0]:
src_index_to_word = get_index_to_word_dict(src_vocab)
dest_index_to_word = get_index_to_word_dict(dest_vocab)

In [129]:
print(src_index_to_word)
print(dest_index_to_word)

{0: 'the', 1: 'treatment', 2: 'of', 3: 'cataract', 4: 'is', 5: 'possible', 6: 'through', 7: 'surgery', 8: 'only', 9: 'complete', 10: 'lens', 11: 'capsule', 12: 'taken', 13: 'out', 14: 'in', 15: 'method', 16: 'intra', 17: 'capsular', 18: 'extraction', 19: 'after', 20: 'operation', 21: 'spectacles', 22: 'are', 23: 'used', 24: 'posttest', 25: 'during', 26: 'implanted', 27: 'at', 28: 'front', 29: 'iris', 30: 'frontal', 31: 'part', 32: 'eye', 33: 'extra', 34: 'and', 35: 'whole', 36: 'matter', 37: 'extracted', 38: 'posterior', 39: 'left', 40: 'intact', 41: 'fitted', 42: 'bag', 43: 's', 44: 'lrb', 45: 'small', 46: 'incision', 47: 'rrb', 48: 'mm', 49: 'tunnel', 50: 'formed', 51: 'while', 52: 'rest', 53: 'follows', 54: 'like', 55: 'that', 56: 'ecceiol', 57: 'no', 58: 'stitches', 59: 'applied', 60: 'black', 61: 'nerves', 62: 'dilapidates', 63: 'gradually', 64: 'it', 65: 'happens', 66: 'due', 67: 'to', 68: 'water', 69: 'pressure', 70: 'or', 71: 'because', 72: 'not', 73: 'enough', 74: 'blood', 75:

In [0]:
def reverse_train_sentences(data):
  l = data.tolist()
  t = []
  for item in l:
    p = item.strip().split()
    p.reverse()
    k = []
    for i in p:
      i = i.strip()
      if len(i) > 0:
        k.append(i)
    p = k
    p = " ".join(p)
    p = p.strip()
    t.append(p)
  df = pd.DataFrame({'data':t})
  return df.data  

In [0]:
def transform(mapping, data, sentence_size):
    transformed_data = np.zeros(shape=(len(data), sentence_size), dtype='float32')
    for i in range(len(data)):
        l = data[i].split()
        for j in range(len(l)):
            transformed_data[i][j] = mapping[l[j]]
    return transformed_data

In [132]:
# Train - Test Split
X, y = lines.src.values, lines.dest.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,shuffle=False)
X_train = reverse_train_sentences(X_train)

encoded_training_input = transform(src_word_to_index, X_train, src_max_len)
encoded_validation_input = transform(src_word_to_index, X_test, src_max_len)
encoded_training_output = transform(dest_word_to_index, y_train, dest_max_len)
encoded_validation_output = transform(dest_word_to_index, y_test, dest_max_len)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)
print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)
# X_train_normal = copy.deepcopy(X_train)
# print(X_train.shape, X_test.shape)
# X_train.sample(10)

Streaming output truncated to the last 5000 lines.
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'panda

In [0]:
latent_dim = 200
embedding_dim = 300

In [134]:
encoder_input = Input(shape=(src_max_len,))
decoder_input = Input(shape=(dest_max_len,))
encoder = Embedding(src_vocab_len, embedding_dim, input_length=src_max_len, mask_zero=True)(encoder_input)
encoder = LSTM(64, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dest_vocab_len, embedding_dim, input_length=dest_max_len, mask_zero=True)(decoder_input)
decoder = LSTM(64, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

encoder Tensor("lstm_4/transpose_2:0", shape=(?, 84, 64), dtype=float32)
encoder_last Tensor("strided_slice_1:0", shape=(?, 64), dtype=float32)
decoder Tensor("lstm_5/transpose_2:0", shape=(?, 86, 64), dtype=float32)


In [135]:
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

output = TimeDistributed(Dense(64, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dest_vocab_len, activation="softmax"))(output)
print('output', output)

attention Tensor("attention_2/truediv:0", shape=(?, 86, 84), dtype=float32)
context Tensor("dot_6/MatMul:0", shape=(?, 86, 64), dtype=float32)
decoder_combined_context Tensor("concatenate_3/concat:0", shape=(?, 86, 128), dtype=float32)
output Tensor("time_distributed_5/Reshape_1:0", shape=(?, 86, 6289), dtype=float32)


In [51]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 86)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 84)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 86, 300)      1414200     input_4[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 84, 300)      1885800     input_3[0][0]                    
__________________________________________________________________________________________

In [0]:

training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = encoding.CHAR_CODE_START
training_decoder_output = np.eye(output_dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = encoding.CHAR_CODE_START
validation_decoder_output = np.eye(output_dict_size)[encoded_validation_output.astype('int')]